In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [14]:
import requests
import pyodbc
import pandas as pd
import sqlalchemy as sa

In [19]:
# Url y header del servicio
url = "https://api.coincap.io/v2/assets"
header = {
    "Content-Type": "application/json",
    "Accept-Encoding": "deflate"
}

In [20]:
# Realizar petición al servicio
response = requests.get(url, headers=header)
responseData = response.json()

In [22]:
# Almacenar datos obtenidos del servicio en DataFrame
df = pd.json_normalize(responseData, "data")

In [23]:
# Obtener primeros 10 registros del DataFrame
df.head(10)

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer
0,bitcoin,1,BTC,Bitcoin,19762906.0000000000000000,21000000.0000000000000000,1218712836442.9899649004362444,19850073178.9086963658744568,61666.6818353024582974,2.1685967624625207,61025.5676151548109902,https://blockchain.info/
1,ethereum,2,ETH,Ethereum,120373387.4931034300000000,None,287733836364.2251237115839004,12455536704.0285344926753565,2390.3442642644770652,2.8063802992822392,2361.3972912079064899,https://etherscan.io/
2,tether,3,USDT,Tether,119641989887.4970600000000000,None,119677872072.9339172339623583,16475596352.9965700682751670,1.0002999129776310,0.0315795314288867,0.9999150222771980,https://www.omniexplorer.info/asset/31
3,binance-coin,4,BNB,BNB,166801148.0000000000000000,166801148.0000000000000000,92027140499.7441095563947500,165462369.9298309815797470,551.7176686322573125,2.3666785842823375,546.2620698692535336,https://etherscan.io/token/0xB8c77482e45F1F44d...
4,solana,5,SOL,Solana,468483517.1755171000000000,None,66480071069.8045001125445325,688952666.5987164265023296,141.9048240386604187,5.5110885857502567,137.9172286304918396,https://explorer.solana.com/
5,usd-coin,6,USDC,USDC,35560914641.7014700000000000,None,35568163560.6475046770354134,2007295261.9949668842363415,1.0002038451209445,0.0295704336745746,1.0002021130773815,https://etherscan.io/token/0xa0b86991c6218b36c...
6,xrp,7,XRP,XRP,45404028640.0000000000000000,100000000000.0000000000000000,23737785817.8851710587441600,516602989.9643842966016819,0.5228123258862690,1.0086481139801442,0.5227450088352403,https://xrpcharts.ripple.com/#/graph/
7,steth,8,STETH,Lido Staked ETH,9807490.2261047800000000,None,23330661745.6360376787358727,298548.8760950157827003,2378.8615851521706693,2.5632346820321215,2346.8076265463953844,https://etherscan.io/token/0xae7ab96520de3a18e...
8,dogecoin,9,DOGE,Dogecoin,146227746383.7052600000000000,None,15772770106.5074394258124473,317076559.7564044826456589,0.1078644135369446,5.0473918761649679,0.1059662802098992,http://dogechain.info/chain/Dogecoin
9,tron,10,TRX,TRON,86589075922.4235000000000000,None,13630415286.4810459500197232,142325414.5068747590691172,0.1574149526516803,1.9541865468396630,0.1565352845638063,https://tronscan.org/#/


In [24]:
# Crear conexión a servidor SQL Server para almacenar datos del servicio
engine = sa.create_engine("mssql+pyodbc://LAPTOP-19URHKFN\\SQLEXPRESS/DWH?driver=SQL+Server+Native+Client+11.0")

In [33]:
# Crear e insertar datos del servicio en una tabla en SQL Server
try:
    df.to_sql(name="FactCrypto", con=engine, index=False, if_exists="fail")
except Exception as error:
  print("Error:", error)

In [34]:
# Consulta en tabla de SQL Server
query = "SELECT * FROM FactCrypto"

# Ejecución de la consulta y carga de resultados en un DataFrame
df_result = pd.read_sql(query, engine)
df_result

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer
0,bitcoin,1,BTC,Bitcoin,19762906.0000000000000000,21000000.0000000000000000,1218712836442.9899649004362444,19850073178.9086963658744568,61666.6818353024582974,2.1685967624625207,61025.5676151548109902,https://blockchain.info/
1,ethereum,2,ETH,Ethereum,120373387.4931034300000000,None,287733836364.2251237115839004,12455536704.0285344926753565,2390.3442642644770652,2.8063802992822392,2361.3972912079064899,https://etherscan.io/
2,tether,3,USDT,Tether,119641989887.4970600000000000,None,119677872072.9339172339623583,16475596352.9965700682751670,1.0002999129776310,0.0315795314288867,0.9999150222771980,https://www.omniexplorer.info/asset/31
3,binance-coin,4,BNB,BNB,166801148.0000000000000000,166801148.0000000000000000,92027140499.7441095563947500,165462369.9298309815797470,551.7176686322573125,2.3666785842823375,546.2620698692535336,https://etherscan.io/token/0xB8c77482e45F1F44d...
4,solana,5,SOL,Solana,468483517.1755171000000000,None,66480071069.8045001125445325,688952666.5987164265023296,141.9048240386604187,5.5110885857502567,137.9172286304918396,https://explorer.solana.com/
...,...,...,...,...,...,...,...,...,...,...,...,...
95,wootrade,96,WOO,WOO,1829264000.1718502000000000,None,307830708.9219135492452903,12115380.7746568666831900,0.1682811824280117,5.6856268977557976,0.1621765653788556,https://etherscan.io/token/0x4691937a7508860f8...
96,amp,97,AMP,Amp,80722574661.2915300000000000,None,305593147.0731205926712224,809528.1191028110459162,0.0037857210124352,4.1076361780782492,0.0037912424086057,None
97,just,98,JST,JUST,9900000000.0000000000000000,9900000000.0000000000000000,283951209.4076879100000000,2741205.2249749122085397,0.0286819403442109,1.2541361355392118,0.0285603630548266,https://tronscan.org/#/token20/TCFLL5dx5ZJdKnW...
98,dash,99,DASH,Dash,11974593.4294003800000000,18900000.0000000000000000,281188051.8641386974852004,32908129.5229596889333096,23.4820541943209012,3.1114301768257827,23.0210985113968742,https://explorer.dash.org


In [35]:
# Crear e insertar datos del servicio en una tabla en SQL Server (Tabla ya existente)
try:
    df.to_sql(name="FactCrypto", con=engine, index=False, if_exists="fail")
except Exception as error:
  print("Error:", error)

Error: Table 'FactCrypto' already exists.
